Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

##  Azure ML and IoT Edge Workshop

For more information go to https://aka.ms/aml-tutorial/ai-iot-power-couple


## 1: Specify parameters
Fill in the parameters below.  If you already have IoT Hub or Azure ML workspace, then enter their information here. Otherwise, the parameter names will be used in provisioning new services.

In [57]:
# Enter the resource group in Azure where you want to provision the resources 
resource_group_name = "rgname"

# Enter the ML ACR Information
mlacr_username = "acruser"
mlacr_password = "acrpasswd"

# Enter Azure region where your services will be provisioned, for example "eastus2"
azure_region="China"

# Enter your Azure IoT Hub name 
# If you don't have an IoT Hub, pick a name to make a new one 
iot_hub_name="hunname"

# Enter your IoT Edge device ID 
# If you don't have an IoT Edge device registered, pick a name to create a new one 
# This is NOT the name of your VM, but it's just an entry in your IoT Hub, so you can pick any name
iot_device_id="devicename"

# Provide your Azure subscription ID to provision your services
subscription_id = "sub_id"

# Provide your Azure ML service workspace name 
# If you don't have a workspace, pick a name to create a new one
aml_workspace_name = "ml_workspace_name"

In [2]:
# DO NOT CHANGE THIS VALUE for this tutorial
# This is the name of the AML module you deploy to the device
module_name = "azuremlad"

In [3]:
!az cloud set -n AzureChinaCloud
!az login

To sign in, use a web browser to open the page https://microsoft.com/deviceloginchina and enter the code DNKPFHCFN to authenticate.
The following tenants don't contain accessible subscriptions. Use 'az login --allow-no-subscriptions' to have tenant level access.
6c42a620-fa0c-400a-88a7-4ef669bb047f
[
  {
    "cloudName": "AzureChinaCloud",
    "homeTenantId": "997f76fe-a833-48b5-b33b-ac2118cc6763",
    "id": "a418906f-b40d-43ee-b86a-ce67f05be91e",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Windows Azure Internal Consumption",
    "state": "Enabled",
    "tenantId": "997f76fe-a833-48b5-b33b-ac2118cc6763",
    "user": {
      "name": "alex@msftiot.partner.onmschina.cn",
      "type": "user"
    }
  }
]


In [5]:
# Load the IoT extension for Azure CLI
!az extension add --name azure-iot

In [6]:
!az account set --subscription $subscription_id
!az iot hub list --output table

Location    Name        Resourcegroup    Subscriptionid
----------  ----------  ---------------  ------------------------------------
chinanorth  IoT-Hub-01  IoT-Mooncake     a418906f-b40d-43ee-b86a-ce67f05be91e


In [7]:
# Check core SDK version number
import azureml.core
from azureml.core import Workspace

print("SDK version:", azureml.core.VERSION)

SDK version: 1.17.0


## 2: Provision Azure ML Workspace and IoT Hub 
If you already have provisioned these resources, then skip this section and go Section 3.

### 2.1: Provision Azure ML workspace (optional)
After you've done this once you no longer have to run this cell anymore

In [ ]:
ws = Workspace.create(subscription_id = subscription_id,
                resource_group = resource_group_name,
                name = aml_workspace_name,
                location = azure_region)

ws.write_config()

### 2.2: Provision IoT Hub (optional)
If you get an error because there's already one free hub in your subscription, change the SKU to S1. If you get an error that the IoT Hub name isn't available, it means that someone else already has a hub with that name. 


In [ ]:
!az iot hub create --resource-group $resource_group_name --name $iot_hub_name --sku F1

In [ ]:
# Register an IoT Edge device (create a new entry in the Iot Hub)
!az iot hub device-identity create --hub-name $iot_hub_name --device-id $iot_device_id --edge-enabled

## 3: Load resources
Load the Azure ML workspace and get the IoT Edge device connection string from your IoT Hub.

### 3.1 Load the Azure ML workspace.

In [11]:
# Initialize a workspace object from persisted configuration
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

ml_workspace
Azure_AI
chinaeast2
a418906f-b40d-43ee-b86a-ce67f05be91e


### 3.2: Get the Azure IoT Edge device connection string

In [16]:
# Get the connection string that you will need to enter in the IoT Edge device
!az iot hub device-identity connection-string show --device-id $iot_device_id --hub-name $iot_hub_name

{
  "connectionString": "HostName=IoT-Hub-01.azure-devices.cn;DeviceId=azure-ai;SharedAccessKey=cVrqjjJrkq6tEVaeNSefdLqkk3UJJ8NyLj1jFowxL14="
}


## 4: Anomaly Detection Model

## 4.1 Train the model

In [17]:
!pip install pandas
!pip install sklearn

In [18]:
# Load the data set
import pandas
import numpy
import pickle


from sklearn import tree
from sklearn.model_selection import train_test_split

In [19]:
temp_data = pandas.read_csv('temperature_data.csv')
temp_data

,machine_temperature,machine_pressure,ambient_temperature,ambient_humidity,anomaly
0,21.018765,1.002138,20.768628,24,0
1,21.965850,1.110034,21.223653,26,0
2,22.999720,1.227816,20.872120,25,0
3,23.731603,1.311195,21.470454,25,0
4,24.902941,1.444639,20.895378,24,0
...,...,...,...,...,...
613,105.877218,10.669556,20.533100,25,1
614,105.990477,10.682459,21.383384,24,1
615,105.642759,10.642846,21.347985,24,1
616,105.742701,10.654232,20.568153,26,1


In [20]:
# Load features and labels
X, Y = temp_data[['machine_temperature', 'machine_pressure', 'ambient_temperature', 'ambient_humidity']].values, temp_data['anomaly'].values

In [21]:
# Split data 65%-35% into training set and test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.35, random_state=0)

# Change regularization rate and you will likely get a different accuracy.
reg = 0.01

# Train a decision tree on the training set
#clf1 = LogisticRegression(C=1/reg).fit(X_train, Y_train)
clf1 = tree.DecisionTreeClassifier()
clf1 = clf1.fit(X_train, Y_train)
print (clf1)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')


In [22]:
# Evaluate the test set
accuracy = clf1.score(X_test, Y_test)

print ("Accuracy is {}".format(accuracy))

Accuracy is 0.9907834101382489


In [23]:
# Serialize the model and write to disk
f = open('model.pkl', 'wb')
pickle.dump(clf1, f)
f.close()
print ("Exported the model to model.pkl")

Exported the model to model.pkl


In [24]:
# Test the model by importing it and providing a sample data point
print("Import the model from model.pkl")
f2 = open('model.pkl', 'rb')
clf2 = pickle.load(f2)

# Normal (not an anomaly)
#X_new = [[24.90294136, 1.44463889, 20.89537849, 24]]
#X_new = [[33.40859853, 2.413637808, 20.89162813, 26]]
#X_new = [[34.42109181, 2.528985143, 21.23903786, 25]]

# Anomaly
X_new = [[33.66995566, 2.44341267, 21.39450979, 26]]
#X_new = [[105.5457931, 10.63179922, 20.62029994, 26]]

print ('New sample: {}'.format(X_new))

pred = clf2.predict(X_new)
print('Predicted class is {}'.format(pred))

Import the model from model.pkl
New sample: [[33.66995566, 2.44341267, 21.39450979, 26]]
Predicted class is [1]


### 4.2 Register Model

You can add tags and descriptions to your models. Note you need to have a `model.pkl` file in the current directory. The below call registers that file as a model with the same name `model.pkl` in the workspace.

Using tags, you can track useful information such as the name and version of the machine learning library used to train the model. Note that tags must be alphanumeric.

In [25]:
from azureml.core.model import Model

model = Model.register(model_path = "model.pkl",
                       model_name = "model.pkl",
                       tags = {'area': "anomaly", 'type': "classification"},
                       description = "Sample anomaly detection model for IOT tutorial",
                       workspace = ws)

Registering model model.pkl


In [26]:
print(model.name, model.description, model.version, sep = '\t')

model.pkl	Sample anomaly detection model for IOT tutorial	1


### 4.2 Create Docker Image

Create the driver file.  Note that the `model.pkl` file in the get_model_path call is referring to a model named `model.pkl` registered under the workspace (what you registered above). It is NOT referencing the local file.

In [27]:
%%writefile iot_score.py
# This script generates the scoring file
# with the init and run functions needed to 
# operationalize the anomaly detection sample

import pickle
import json
import pandas
import joblib
from sklearn.linear_model import Ridge
from azureml.core.model import Model

def init():
    global model
    # this is a different behavior than before when the code is run locally, even though the code is the same.
    model_path = Model.get_model_path('model.pkl')
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)

# note you can pass in multiple rows for scoring
def run(input_str):
    try:
        input_json = json.loads(input_str)
        input_df = pandas.DataFrame([[input_json['machine']['temperature'],input_json['machine']['pressure'],input_json['ambient']['temperature'],input_json['ambient']['humidity']]])
        pred = model.predict(input_df)
        print("Prediction is ", pred[0])
    except Exception as e:
        result = str(e)
        
    if pred[0] == 1:
        input_json['anomaly']=True
    else:
        input_json['anomaly']=False
        
    return [json.dumps(input_json)]


Overwriting iot_score.py


In [28]:
# This specifies the dependencies to include in the environment
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(conda_packages=['pandas', 'scikit-learn', 'numpy'])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

You can add tags and descriptions to images. Also, an image can contain multiple models.

In [32]:
from azureml.core.image import Image, ContainerImage

image_config = ContainerImage.image_configuration(runtime= "python",
                                 execution_script="iot_score.py",
                                 conda_file="myenv.yml",
                                 tags = {'area': "iot", 'type': "anamaly-classification"},
                                 description = "IOT Edge anomaly detection demo")


image = Image.create(name = "tempanomalydetection",
                     # this is the model object 
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: ContainerImage class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  import sys
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Image class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  


Creating image


Note that following command can take few minutes. 

In [33]:
image.wait_for_creation(show_output = True)

Running...........................................................................................
Succeeded
Image creation operation finished for image tempanomalydetection:2, operation "Succeeded"


List images by tag and find out the detailed build log for debugging.

In [31]:
for i in Image.list(workspace = ws,tags = ["area"]):
    print('{}(v.{} [{}]) stored at {} with build log {}'.format(i.name, i.version, i.creation_state, i.image_location, i.image_build_log_uri))

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Image class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  """Entry point for launching an IPython kernel.


tempanomalydetection(v.1 [Succeeded]) stored at 19002c4599604129be865e1586c0287b.azurecr.cn/tempanomalydetection:1 with build log https://mlworkspace1842961028.blob.core.chinacloudapi.cn/azureml/ImageLogs/57f3f19b-adb4-456a-824e-024da79b2f93/build.log?sv=2019-02-02&sr=b&sig=EiuwbjTHGZ8a7QbV2dxqB0UvWXez0PBRjjV6OcInCzY%3D&st=2021-05-30T04%3A55%3A06Z&se=2021-06-29T05%3A00%3A06Z&sp=rl


## 5: Test model on Azure Container Instance
You can test the performance of your model by deploying the container to ACI.  Note that the service creation can take few minutes.

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'area': "iot", 'type': "classification"}, 
                                               description = 'IOT Edge anomaly detection demo')

In [ ]:
from azureml.core.webservice import Webservice

aci_service_name = 'tempsensor-iotedge-ml-1'
print(aci_service_name)
aci_service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                           image = image,
                                           name = aci_service_name,
                                           workspace = ws)
aci_service.wait_for_deployment(True)
print(aci_service.state)

### 5.1: Test web service

Call the web service with some dummy input data to get a prediction.

In [ ]:
import json

# Anomaly
#test_sample = json.dumps({ "machine": { "temperature": 33.66995566, "pressure": 2.44341267 }, \
#                          "ambient": { "temperature": 21.39450979, "humidity": 26 },\
#                          "timeCreated": "2017-10-27T18:14:02.4911177Z" })

# Normal
test_sample = json.dumps({ "machine": { "temperature": 31.16469009, "pressure": 2.158002669 }, \
                          "ambient": { "temperature": 21.17794693, "humidity": 25 },\
                          "timeCreated": "2017-10-27T18:14:02.4911177Z" })

test_sample = bytes(test_sample,encoding = 'utf8')

prediction = aci_service.run(input_data = test_sample)
print(prediction)

### 5.2: Delete ACI to clean up

In [ ]:
aci_service.delete()

## 6: Deploy container to Azure IoT Edge device
Create a deployment.json file that contains the modules you want to deploy to the device and the routes.  Then push this file to the IoT Hub, which will then send it to the IoT Edge device.  The IoT Edge agent will then pull the Docker images and run them.

This section pulls the container registry information from your workspace. Machine Learning workspaces don't automatically provision container registries when they're created, but wait until there's a container image to store. In the case of this tutorial, the container image was created in section 4.2. So in this section, we update the workspace config information. Your container registry still may not be ready by the time you get to this step. Keep running the first cell of this section until it finds the registry information. You can also check to see if the container registry exists yet by viewing the workspace overview page in the Azure portal.

In [55]:
# Update the workspace object
ws = Workspace.from_config()

# Getting your container details
container_reg = ws.get_details()["containerRegistry"]
reg_name=container_reg.split("/")[-1]
container_url = "\"" + image.image_location + "\","
subscription_id = ws.subscription_id
print('{}'.format(image.image_location))
print('{}'.format(reg_name))
print('{}'.format(subscription_id))

username = mlacr_username
password = mlacr_password

19002c4599604129be865e1586c0287b.azurecr.cn/tempanomalydetection:2
19002c4599604129be865e1586c0287b
a418906f-b40d-43ee-b86a-ce67f05be91e


In [59]:
file = open('iot-workshop-deployment-template.json')
contents = file.read()
contents = contents.replace('__MODULE_NAME', module_name)
contents = contents.replace('__REGISTRY_NAME', reg_name)
contents = contents.replace('__REGISTRY_USER_NAME', username)
contents = contents.replace('__REGISTRY_PASSWORD', password)
contents = contents.replace('__REGISTRY_IMAGE_LOCATION', image.image_location)
with open('./deployment.json', 'wt', encoding='utf-8') as output_file:
    output_file.write(contents)

In [ ]:
# Push the deployment JSON to the IOT Hub
!az iot edge set-modules --device-id $iot_device_id --hub-name $iot_hub_name --content deployment.json

## Congratulations!
You made it to the end of the tutorial!  You can monitor messages from your edge device to your IoT Hub with VS Code and the [Azure IoT Hub Toolkit](https://marketplace.visualstudio.com/items?itemName=vsciot-vscode.azure-iot-toolkit) extension.